#Params

In [1]:
DATA_SOURCE: str = 'https://github.com/h4438/data/raw/master/daily_articles.csv'
DATA_TYPE: str = 'csv'
MODEL_NAME: str = 'bert-base-multilingual-cased'
GDRIVE_PATH: str = 'gdrive/MyDrive/models/'

#Import

In [ ]:
# https://stackoverflow.com/questions/39142778/how-to-determine-the-language-of-a-piece-of-text
# https://www.geeksforgeeks.org/how-to-count-occurrences-of-specific-value-in-pandas-column/
# https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
from tqdm.auto import tqdm
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from transformers import BertTokenizer
import os
from google.colab import drive, files

##bert process

In [4]:
class BERT_Classifier:
  
  MODELS = {'bert-base-multilingual-cased':'vnnews_multi_bert'}
  
  def __init__(self, nlabels = 3):
    print("Classifier is ready")
    self.nlabels = nlabels
    return

  def DatasetMapFunction(self, input_ids, attn_masks, labels):
    # define the output of the whole
    return {
        'input_ids':input_ids,
        'attention_mask':attn_masks},labels

  def load_model_tokenizer(self, MODEL_NAME):
    self.tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
    bert_path = GDRIVE_PATH+BERT_Classifier.MODELS[MODEL_NAME]
    self.bert = tf.keras.models.load_model(bert_path)
    return

  def tokenize_data(self, data,max_len=256):
    ids = np.zeros((len(data),max_len))
    masks = np.zeros((len(data),max_len))
    for i, text in tqdm(enumerate(data)):
      tokenized_text = self.tokenizer.encode_plus(
          text,
          max_length=max_len,
          truncation=True,
          padding='max_length',
          add_special_tokens=True,
          return_tensors='tf'
      )
      ids[i,:] = tokenized_text.input_ids
      masks[i,:] = tokenized_text.attention_mask
    return ids, masks


  def label_news(self, text_data):
    # input
    text = text_data
    # tokenize
    x_input_ids, x_attn_masks = self.tokenize_data(text)
    # one hot
    labels = np.zeros((len(text),self.nlabels))
    # create dataset
    dataset = tf.data.Dataset.from_tensor_slices(
      (x_input_ids, x_attn_masks, labels))
    # change format
    dataset = dataset.map(self.DatasetMapFunction)
    text_ds = dataset.batch(1)
    pred_raw = self.bert.predict(text_ds)
    pred = np.argmax(pred_raw, axis=1)
    pred = pred+1 # due to index transition
    return text,dataset,pred

##data process

In [8]:
class News_Process:
  def __init__(self):
    print("Ready to process news")
    self.origin_df = None
    self.origin_langs = None
    return
  
  def read_news(self, src = DATA_SOURCE, type = DATA_TYPE):
    if DATA_TYPE == 'csv':
      self.origin_df = pd.read_csv(src)
    else:
      self.origin_df = pd.read_excel(src)
    return self.origin_df

  def visualize_news(self):
    ntc = "\n--->"
    print(self.origin_df.head(5))
    print(f'{ntc}Data shape',self.origin_df.shape)
    print(f'{ntc}Unique values of each column')
    print(self.origin_df.nunique())
    print(f'{ntc}Nan values of each column')
    print(self.origin_df.isna().sum())
    return
    
  def handle_nan_of(self, target_col, value):
    self.origin_df[target_col]=self.origin_df[target_col].fillna(value)
    return

  def lower_case(self, target_col = ['title']):
    for col in target_col:
      self.origin_df[col]=self.origin_df[col].str.lower()
    return

  def remove_non_ascii(self, target_col = ['title']):
    for col in target_col:
      self.origin_df[col]=\
      self.origin_df[col].str.encode('ascii','ignore').str.decode('ascii')
    return

  def combine_text(self, target_col, out_col='combine'):
    data = self.origin_df[target_col[0]]
    for col in target_col[1:]:
      data += (" "+self.origin_df[col])
    self.origin_df[out_col] = data
    return

  def drill_down(self,target_col):
    res = self.origin_df[target_col].value_counts()
    print(res)
    return 
  
  def get_df(self):
    return self.origin_df

  def set_df(self, df):
    self.origin_df = df
    return

  def add_column(self,data,name):
    self.origin_df[name] = data
    return

  

In [ ]:
df = pd.Series(['m©ª«zy', '¤¥uw', 'ÆÇva%l 672'])
d= df.str.encode('ascii', 'ignore').str.decode('ascii')
print("After removing non-ascii:")
print(d)

After removing non-ascii:
0         mzy
1          uw
2    va%l 672
dtype: object


##load from GDrive

In [6]:
drive.mount("gdrive")

Mounted at gdrive


In [ ]:
os.getcwd()

'/content'

#Data

In [9]:
txt_new = ['title','description']
news_ps = News_Process()
news_ps.read_news()
news_ps.lower_case(txt_new)
news_ps.visualize_news()

Ready to process news
         date                                              title  \
0  2023-01-18                người dùng chuộng thanh toán online   
1  2023-01-23  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...   
2  2023-01-13  vietnam international commercialbank : payment...   
3  2023-01-12  vn-index continues sideways - vnexpress intern...   
4  2023-01-12  nhiều hoạt động tương tác và quà tặng tại momo...   

                                         description stock_group  
0  nhiều người chọn mua sắm, thanh toán với thẻ t...         VIB  
1  thẻ vib travel elite giúp tích lũy dặm thưởng,...         VIB  
2  (marketscreener.com) \n \n vietnam internation...         VIB  
3  vietnam’s benchmark vn-index rose 0.06% to 1,0...         VIB  
4  thông qua lắc xì 2023 với nhiều quà tặng giá t...         VIB  

--->Data shape (1781, 4)

--->Unique values of each column
date            33
title          523
description    529
stock_group     33
dtype: int64

--->Nan values of

In [11]:
news_ps.handle_nan_of('description',"")
news_ps.combine_text(txt_new)
news_ps.visualize_news()
in_data = news_ps.get_df()

         date                                              title  \
0  2023-01-18  người dùng chuộng thanh toán online nhiều ngườ...   
1  2023-01-23  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...   
2  2023-01-13  vietnam international commercialbank : payment...   
3  2023-01-12  vn-index continues sideways - vnexpress intern...   
4  2023-01-12  nhiều hoạt động tương tác và quà tặng tại momo...   

                                         description stock_group  \
0  nhiều người chọn mua sắm, thanh toán với thẻ t...         VIB   
1  thẻ vib travel elite giúp tích lũy dặm thưởng,...         VIB   
2  (marketscreener.com) \n \n vietnam internation...         VIB   
3  vietnam’s benchmark vn-index rose 0.06% to 1,0...         VIB   
4  thông qua lắc xì 2023 với nhiều quà tặng giá t...         VIB   

                                             combine  
0  người dùng chuộng thanh toán online nhiều ngườ...  
1  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...  
2  vietnam intern

#Model

In [12]:
my_bert = BERT_Classifier()
my_bert.load_model_tokenizer(MODEL_NAME)

Classifier is ready


In [13]:
sample = in_data.iloc[:10,:]
print(sample.shape)
print(in_data.shape)
print(in_data.head())

(10, 5)
(1781, 5)
         date                                              title  \
0  2023-01-18  người dùng chuộng thanh toán online nhiều ngườ...   
1  2023-01-23  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...   
2  2023-01-13  vietnam international commercialbank : payment...   
3  2023-01-12  vn-index continues sideways - vnexpress intern...   
4  2023-01-12  nhiều hoạt động tương tác và quà tặng tại momo...   

                                         description stock_group  \
0  nhiều người chọn mua sắm, thanh toán với thẻ t...         VIB   
1  thẻ vib travel elite giúp tích lũy dặm thưởng,...         VIB   
2  (marketscreener.com) \n \n vietnam internation...         VIB   
3  vietnam’s benchmark vn-index rose 0.06% to 1,0...         VIB   
4  thông qua lắc xì 2023 với nhiều quà tặng giá t...         VIB   

                                             combine  
0  người dùng chuộng thanh toán online nhiều ngườ...  
1  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...  

In [ ]:
in_data.iloc[240,:]

date                                                  2023-01-10
title          philippines sea exploration deal with beijing,...
description                                                     
stock_group                                                  BID
combine        philippines sea exploration deal with beijing,...
Name: 240, dtype: object

In [20]:
origin_news, dataset, labels = my_bert.label_news(in_data['title'])

0it [00:00, ?it/s]

1781/1781 [==============================] - 2043s 1s/step


In [21]:
a = [print(f'{text[:110]}...:{pds}') for text,pds in zip(origin_news,labels)]
b = 1

người dùng chuộng thanh toán online nhiều người chọn mua sắm, thanh toán với thẻ tín dụng vib để hưởng bộ ba ư...:2
thẻ tín dụng vib travel elite nhiều ưu đãi dịp tết thẻ vib travel elite giúp tích lũy dặm thưởng, đặc quyền ph...:3
vietnam international commercialbank : payment instruction for deposit book having due date during tet holiday...:2
vn-index continues sideways - vnexpress international vietnam’s benchmark vn-index rose 0.06% to 1,056.39 poin...:3
nhiều hoạt động tương tác và quà tặng tại momo lắc xì 2023 thông qua lắc xì 2023 với nhiều quà tặng giá trị, m...:2
bustling m&a deals in banking industry at year beginning right from the beginning of 2023, many banks have ann...:2
banks plan to pay cash dividends for first time in three years some banks have announced plans to pay cash div...:3
người dùng chuộng thanh toán online nhiều người chọn mua sắm, thanh toán với thẻ tín dụng vib để hưởng bộ ba ư...:2
thẻ tín dụng vib travel elite nhiều ưu đãi dịp tết thẻ vib travel elite 

#Result

In [22]:
in_data['title_labels'] = labels

In [23]:
in_data.to_csv("newsapi_articles.csv")

In [25]:
process = News_Process()
process.set_df(in_data)
process.visualize_news()
print("\n--> Looking into full_labels")
process.drill_down("full_labels")
process.drill_down("title_labels")

Ready to process news
         date                                              title  \
0  2023-01-18  người dùng chuộng thanh toán online nhiều ngườ...   
1  2023-01-23  thẻ tín dụng vib travel elite nhiều ưu đãi dịp...   
2  2023-01-13  vietnam international commercialbank : payment...   
3  2023-01-12  vn-index continues sideways - vnexpress intern...   
4  2023-01-12  nhiều hoạt động tương tác và quà tặng tại momo...   

                                         description stock_group  \
0  nhiều người chọn mua sắm, thanh toán với thẻ t...         VIB   
1  thẻ vib travel elite giúp tích lũy dặm thưởng,...         VIB   
2  (marketscreener.com) \n \n vietnam internation...         VIB   
3  vietnam’s benchmark vn-index rose 0.06% to 1,0...         VIB   
4  thông qua lắc xì 2023 với nhiều quà tặng giá t...         VIB   

                                             combine  full_labels  \
0  người dùng chuộng thanh toán online nhiều ngườ...            2   
1  thẻ tín dụng vib tr

In [ ]:
b = 1

2    1423
3     300
1      58
Name: full_labels, dtype: int64
